<a href="https://colab.research.google.com/github/JoySoon/Python_Study/blob/main/%EA%B3%BC%EC%A0%9C/XGBoost_KSW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import files

files.upload()

In [8]:
bd=pd.read_csv('cbb.csv')
bd

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,...,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED,YEAR
0,North Carolina,ACC,40,33,123.3,94.9,0.9531,52.6,48.1,15.4,...,30.4,53.9,44.6,32.7,36.2,71.7,8.6,2ND,1.0,2016
1,Wisconsin,B10,40,36,129.1,93.6,0.9758,54.8,47.7,12.4,...,22.4,54.8,44.7,36.5,37.5,59.3,11.3,2ND,1.0,2015
2,Michigan,B10,40,33,114.4,90.4,0.9375,53.9,47.7,14.0,...,30.0,54.7,46.8,35.2,33.2,65.9,6.9,2ND,3.0,2018
3,Texas Tech,B12,38,31,115.2,85.2,0.9696,53.5,43.0,17.7,...,36.6,52.8,41.9,36.5,29.7,67.5,7.0,2ND,3.0,2019
4,Gonzaga,WCC,39,37,117.8,86.3,0.9728,56.6,41.1,16.2,...,26.9,56.3,40.0,38.2,29.0,71.5,7.7,2ND,1.0,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2450,Michigan St.,B10,35,26,111.4,87.8,0.9392,50.6,44.5,20.8,...,32.4,50.4,44.3,34.1,30.1,64.4,6.7,S16,3.0,2013
2451,Arizona,P12,35,27,114.4,92.2,0.9229,52.5,46.6,19.5,...,32.9,50.6,43.4,37.1,35.8,66.8,4.6,S16,6.0,2013
2452,Oregon,P12,37,28,104.8,88.6,0.8728,49.3,46.4,21.4,...,33.3,49.1,44.9,33.3,33.4,69.2,2.9,S16,12.0,2013
2453,La Salle,A10,34,24,112.0,96.2,0.8516,51.9,49.3,17.1,...,28.5,49.3,50.6,37.7,30.2,66.0,0.3,S16,13.0,2013


In [ ]:
'''
TEAM : 참여하는 학교의 이름
CONF : 소속 지역
G : 게임수
W : 승리한 게임수
ADJOE : 조정된 공격 효율성(평균 디비전 I 방어에 대해 팀이 가질 공격 효율성(점유율당 득점)의 추정치)
ADJDE : 수정된 방어 효율성(평균 디비전 I 공격에 대해 팀이 가질 방어 효율성(점유율당 실점)의 추정치)
BARTHAG : 전력 등급(평균 디비전 I 팀을 이길 가능성)
EFG_O : 유효슛 비율
EFG_D : 유효슛 허용 비율
TOR : 턴오버 비율(흐름 끊은 비율)
TORD : 턴오버 허용 비율(흐름 끊긴 비율)
ORB : 리바운드 차지 횟수
DRB : 리바운드 허용 횟수
FTR : 자유투 비율
FTRD : 자유투 허용 비율
2P_O : 2점 슛 성공 비율
2P_D : 2점 슛 허용 비율
3P_O : 3점 슛 성공 비율
3P_D : 3점 슛 허용 비율
ADJ_T : 조정된 템포(팀이 평균 디비전 I 템포로 플레이하려는 팀을 상대로 가질 템포(40분당 점유)의 추정치)
WAB : "Wins Above Bubble"은 NCAA 농구 대회의 예선 라운드에 참가하는 팀을 결정하는 데 사용되는 "버블"(일정 선) 기준에서 얼마나 높은 승리를 거두었는지를 나타내는 지표입니다.
POSTSEASON : 팀이 시즌을 마무리한 등수
SEED : NCAA 토너먼트에 참가하는 시드(등수)
YEAR : 시즌

'''

In [9]:
bd["Winning_rate"] = bd['W']/bd['G']

#R68 = 퍼스트 포, R64 = 64강, R32 = 32강, S16 = 스위트 16, E8 = 엘리트 에이트, F4 = 파이널 포, 2ND = 러너 -up, Champion = 해당 연도의 NCAA March Madness Tournament 우승자
ps={
    "R68":68,
    "R64":64,
    "R32":32,
    "S16":16,
    "E8":8,
    "F4":4,
    "2ND":2,
    "Champion":1
}

#postseason의 결측치 처리
bd['POSTSEASON'] = bd['POSTSEASON'].map(ps)

In [12]:
bd.fillna({'POSTSEASON':'Missed Tournament'}, inplace = True)
bd.fillna({'SEED':'Missed Tournament'}, inplace = True)

In [11]:
bd=pd.get_dummies(bd, columns=['CONF','SEED','POSTSEASON'])  

In [13]:
bd.drop(['TEAM', 'YEAR','W','G'], axis=1, inplace=True)

In [ ]:
for i in bd:

  sns.scatterplot(bd[i])
  plt.show()

In [15]:
bd.corr()['Winning_rate'].sort_values()[:]

ADJDE                          -0.652915
EFG_D                          -0.571571
SEED_Missed Tournament         -0.570116
POSTSEASON_Missed Tournament   -0.558596
2P_D                           -0.500915
                                  ...   
EFG_O                           0.625103
ADJOE                           0.718785
BARTHAG                         0.788895
WAB                             0.888216
Winning_rate                    1.000000
Name: Winning_rate, Length: 78, dtype: float64

# 학습/시험 셋 분리 및 스케일링

In [16]:
from sklearn.model_selection import train_test_split

X_xgb = bd.drop('Winning_rate', axis=1)
y_xgb = bd['Winning_rate']

X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(
     X_xgb, y_xgb, test_size=0.2, random_state=325 
)

In [17]:
# min-max 스케일링
from sklearn.preprocessing import MinMaxScaler
mm_scaler = MinMaxScaler()
X_train_scaled = mm_scaler.fit_transform(X_train_xgb)
X_test_scaled = mm_scaler.transform(X_test_xgb)

# XGBoost

In [18]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [23]:
xg_reg = xgb.XGBRegressor(
    objective ='reg:squarederror', 
    max_depth = 3, 
    learning_rate = 0.1, 
    n_estimators = 10,
    subsample = 0.5, 
    colsample_bytree = 0.3, 
)

# Train the model on the training data
xg_reg.fit(X_train_xgb, y_train_xgb)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.3, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=10, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [28]:
y_pred_xgb = xg_reg.predict(X_test_xgb)

rmse = np.sqrt(mean_squared_error(y_test_xgb, y_pred_xgb))
print("RMSE: %f" % (rmse))

RMSE: 0.119270


In [25]:
xg_reg.score(X_test_xgb, y_test_xgb)

# XGBRegressor 모델이 비교적 잘 수행되고 있지만 개선의 여지가 있음을 나타냅니다.

0.6018204222637287

# 하이퍼 파라미터 적용

In [22]:
from sklearn.model_selection import GridSearchCV

In [27]:
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [10, 50, 100],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.3, 0.5, 1.0]
}

grid_search = GridSearchCV(xg_reg, param_grid=param_grid, cv=5, n_jobs=-1)

grid_search.fit(X_xgb, y_xgb)

# Print the best hyperparameters and corresponding score
print("Best hyperparameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

Best hyperparameters: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.5}
Best score: 0.889459752872298


# 최고의 점수값 적용

In [33]:
xg_reg_hp = xgb.XGBRegressor(
    colsample_bytree = 1.0,
    learning_rate=0.1,
    max_depth=5,
    n_estimators=100,
    subsample=0.5,
    n_jobs=-1,
    random_state=325
)

xg_reg_hp.fit(X_train_xgb, y_train_xgb)
train_pred_xgb = xg_reg_hp.predict(X_train_xgb)
test_pred_xgb = xg_reg_hp.predict(X_test_xgb)

xg_reg_hp.score(X_test_xgb, y_test_xgb)

0.9354254445716658

# pkl파일

In [34]:
!pip install mlxtend -q

In [35]:
import joblib

In [36]:
joblib.dump(xg_reg, 'XGBoost.pkl')

['XGBoost.pkl']

In [37]:
files.download('XGBoost.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
model_from_joblib = joblib.load('XGBoost.pkl')
model_from_joblib

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.3, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=10, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)